In [1]:
import pyodbc
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import joblib
import scipy
from sqlalchemy import create_engine

In [2]:
# Kết nối Azure SQL Server
server = 'is402server.database.windows.net'
database = 'is402db'
username = 'trieu'
password = 'Admin003@'
driver = '{ODBC Driver 18 for SQL Server}'

try:
    conn = pyodbc.connect(
        f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}'
    )
    print("Kết nối đến Azure SQL Server thành công!")
except Exception as e:
    print(f"Lỗi kết nối đến SQL Server: {e}")
    exit()

# Truy vấn dữ liệu
reviews_query = """
SELECT r.text, r.stars AS review_stars, b.categories, b.stars AS business_stars, b.review_count
FROM fact_review r
JOIN dim_business b ON r.business_id = b.business_id
"""
try:
    df = pd.read_sql(reviews_query, conn)
    print(f"Đọc thành công {len(df)} dòng dữ liệu!")
except Exception as e:
    print(f"Lỗi truy vấn dữ liệu: {e}")
    exit()


Kết nối đến Azure SQL Server thành công!


C:\Users\trieu\AppData\Local\Temp\ipykernel_22636\2244146285.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(reviews_query, conn)


Đọc thành công 6990247 dòng dữ liệu!


In [3]:
def preprocess_text(text):
    import re
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ ký tự đặc biệt
    text = text.lower()  # Chuyển về chữ thường
    return text

df['text_clean'] = df['text'].apply(preprocess_text)

# Nhãn: Chuyển số sao thành hai nhóm
df['label'] = df['review_stars'].apply(lambda x: 1 if x >= 4 else 0)

In [4]:
# Đặc trưng: Văn bản và các thuộc tính bổ trợ
X_text = df['text_clean']
X_features = df[['business_stars', 'review_count']]
y = df['label']

# Chuẩn hóa các thuộc tính bổ trợ
scaler = StandardScaler()
X_features_scaled = scaler.fit_transform(X_features)

# Trích xuất đặc trưng từ văn bản (TF-IDF)
tfidf = TfidfVectorizer(max_features=5000)  # Chọn 5000 đặc trưng phổ biến nhất
X_tfidf = tfidf.fit_transform(X_text)

# Kết hợp đặc trưng từ TF-IDF và các thuộc tính bổ trợ
X_combined = scipy.sparse.hstack((X_tfidf, X_features_scaled))

# Chia dữ liệu train/test
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Huấn luyện Logistic Regression
model = LogisticRegression(max_iter=5000, solver='saga')
try:
    model.fit(X_train, y_train)
    print("Huấn luyện mô hình thành công!")
except Exception as e:
    print(f"Lỗi trong quá trình huấn luyện mô hình: {e}")
    exit()

Huấn luyện mô hình thành công!


In [5]:
# Đánh giá mô hình
y_pred = model.predict(X_test)
print("Báo cáo đánh giá mô hình:")
print(classification_report(y_test, y_pred))

# Lưu mô hình và TF-IDF
try:
    joblib.dump(model, 'recommendation_model.pkl')
    joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
    print("Lưu mô hình và vectorizer thành công!")
except Exception as e:
    print(f"Lỗi khi lưu mô hình: {e}")

Báo cáo đánh giá mô hình:
              precision    recall  f1-score   support

           0       0.89      0.86      0.88    460473
           1       0.93      0.95      0.94    937577

    accuracy                           0.92   1398050
   macro avg       0.91      0.91      0.91   1398050
weighted avg       0.92      0.92      0.92   1398050

Lưu mô hình và vectorizer thành công!
